In [1]:
import numpy as np
import pandas as pd

In [2]:
# Считываем входные данные
data = pd.read_csv('input.csv', skipinitialspace=True)
data['prev'] = data['prev'].fillna(value='-')
data

,name,duration,prev
0,W0,15,-
1,W1,9,-
2,W2,12,-
3,W3,11,W0 W1
4,W4,17,W0 W1
5,W5,6,W0
6,W6,16,W3 W4 W5
7,W7,16,W4 W5 W2
8,W8,10,W6 W7
9,W9,5,W7


In [4]:
# Листы, куда будем складывать входные и выходные точки
inputs_list = []
outputs_list = []

# Формируем граф в виде матрицы смежности
# Где нет ребра, там -1
# Нулевая вершина - начало, последняя - конец
# Для i-й работы начало - 2i+1, конец 2i+2
adj_matr = np.full([data.shape[0]*2 + 2, data.shape[0]*2 + 2], -1)
for idx, [name, duration, prev_str] in data.iterrows():
    # Работа - ребро между своим началом и концом
    adj_matr[idx*2+1, idx*2+2] = duration
    # Если нет предыдущих, то начало этой работы - вход
    if prev_str == '-':
        inputs_list.append(idx*2+1)
        adj_matr[0, idx*2+1] = 0;
    # Иначе связываем концы предыдущих работ с началом этой
    else:
        for name_prev in prev_str.split():
            temp_df = data[data['name'] == name_prev]
            if temp_df.size != 0:
                idx_to = temp_df.index[0]
                adj_matr[idx_to*2+2, idx*2+1] = 0
            
# Проходимся по концам работ, если нет дуги - это выход
for idx in range(1, data.shape[0]):
    if np.all(adj_matr[idx*2+2] == -1):
        outputs_list.append(idx*2+2)
        adj_matr[idx*2+2, -1] = 0;
        
print('Входные вершины:', inputs_list)
print('Выходные вершины:', outputs_list)

Входные вершины: [1, 3, 5]
Выходные вершины: [18, 20]


In [5]:
print(adj_matr)

[[-1  0 -1  0 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 15 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1  0 -1  0 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1  9 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1  0 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 12 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 11 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 17 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1  0 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  6 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1  0 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 16 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1

In [8]:
# К этому моменту граф должен быть топологически отсортирован
# Т.е. все дуги выходят из вершин с меньшими индексами
# И входят в вершины с большими


# Проходим от начала до конца, находим максимальные пути
distances = np.full(data.shape[0]*2 + 2, -1)
distances[0] = 0;
# Смотрим по очереди все вершины
for i in range(data.shape[0]*2 + 2):
    # Путь до текущей берем как максимальную сумму
    # предыдущего пути + ребра
    for j in range(i):
        if (adj_matr[j, i] != -1) and (distances[j] + adj_matr[j, i] > distances[i]):
            distances[i] = distances[j] + adj_matr[j, i]

# dist_forward_df = pd.DataFrame({'vertex' : range(distances.shape[0]), 'max distance' : distances})
# dist_forward_df['max distance']
print(distances)

[ 0  0 15  0  9  0 12 15 26 15 32 15 21 32 48 32 48 48 58 48 53 58]


In [9]:
# Проходим из конца в начало
# Смотрим по очереди все вершины
distances_b = np.full(data.shape[0]*2 + 2, distances[-1])
for i in reversed(range(data.shape[0]*2 + 2)):
    # Путь до текущей берем как минимум
    # предыдущего пути и ребра
    for j in range(i, data.shape[0]*2 + 2):
        if (adj_matr[i, j] != -1) and (distances[j] - adj_matr[i, j] < distances_b[i]):
            distances_b[i] = distances_b[j] - adj_matr[i, j]
            
# dist_backward_df = pd.DataFrame({'vertex' : range(distances_b.shape[0]), 'max distance' : distances_b})
# dist_backward_df['max distance']
print(distances_b)

[ 0  0 15  6 15 20 32 21 32 15 32 26 32 32 48 32 48 48 58 53 58 58]


In [10]:
# Выходные данные
e_sf = distances[1: -1].reshape([data.shape[0], 2])
l_sf = distances_b[1: -1].reshape([data.shape[0], 2])

out = pd.DataFrame({'name' : data['name'],
                    'duration' : data['duration'],
                    'early start' : e_sf[:, 0],
                    'late start' : l_sf[:, 0],
                    'early finish' : e_sf[:, 1],
                    'late finish' : l_sf[:, 1],
                    'time margin' : l_sf[:, 0] - e_sf[:, 0]})

out.to_csv('out.csv', index=False)
out

,name,duration,early start,late start,early finish,late finish,time margin
0,W0,15,0,0,15,15,0
1,W1,9,0,6,9,15,6
2,W2,12,0,20,12,32,20
3,W3,11,15,21,26,32,6
4,W4,17,15,15,32,32,0
5,W5,6,15,26,21,32,11
6,W6,16,32,32,48,48,0
7,W7,16,32,32,48,48,0
8,W8,10,48,48,58,58,0
9,W9,5,48,53,53,58,5


In [11]:
# Критический путь
crit_path_full = out[out['early start'] == out['late start']]
crit_path = pd.DataFrame({'name' : crit_path_full['name'],
                          'duration' : crit_path_full['duration'],
                          'start' : crit_path_full['early start'],
                          'finish' : crit_path_full['early finish']})

crit_path.to_csv('crit_path.csv', index=False)
crit_path

,name,duration,start,finish
0,W0,15,0,15
4,W4,17,15,32
6,W6,16,32,48
7,W7,16,32,48
8,W8,10,48,58
